In [1]:
import numpy as np
import cv2

In [ ]:
imagen = cv2.imread("../../../")

In [ ]:
# -*- coding: utf-8 -*-
"""
Genera dos GIF pedagógicos que ilustran la convolución en 1D y 2D.

Requisitos:
  pip install numpy matplotlib imageio scipy

Salida:
  - convolucion_1d.gif
  - convolucion_2d.gif
"""

import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from scipy.signal import convolve, convolve2d
from matplotlib.patches import Rectangle

# ---------- Utilidades ----------
def fig_to_rgb_array(fig):
    """
    Renderiza una figura de matplotlib a un array RGB (uint8),
    compatible con backends que exponen buffer_rgba() o tostring_rgb().
    """
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    if hasattr(fig.canvas, "buffer_rgba"):
        buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        arr = buf.reshape((h, w, 4))[:, :, :3]  # descarta canal alpha
        return arr
    elif hasattr(fig.canvas, "tostring_rgb"):
        buf = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        return buf.reshape((h, w, 3))
    else:
        # Fallback robusto vía PNG en memoria (más lento, pero seguro)
        import io, imageio.v2 as imageio
        mem = io.BytesIO()
        fig.savefig(mem, format="png", dpi=fig.dpi, bbox_inches="tight")
        mem.seek(0)
        im = imageio.imread(mem)
        mem.close()
        return im[:, :, :3]

def normalize01(x, eps=1e-12):
    x = np.asarray(x, dtype=np.float32)
    mn = np.min(x)
    mx = np.max(x)
    return (x - mn) / (mx - mn + eps)

# ---------- GIF de Convolución 1D ----------
def generar_gif_convolucion_1d(
    n=120,
    kernel=None,
    fps=20,
    archivo_salida="convolucion_1d.gif",
    dpi=120,
):
    # Señal 1D sintética: suma de senos + pulsos
    x = np.linspace(0, 4*np.pi, n)
    señal = 0.7*np.sin(1.0*x) + 0.3*np.sin(3.2*x)
    señal += np.exp(-0.5*((x-2.2*np.pi)/0.25)**2)  # pulso gaussiano
    señal += 0.6*np.exp(-0.5*((x-0.8*np.pi)/0.12)**2)

    # Kernel: si no se pasa, usar un suavizante triangular (ventana Bartlett)
    if kernel is None:
        ksize = 15
        kernel = np.bartlett(ksize)
        kernel /= np.sum(kernel)

    # Convolución "full" para referencia y escala
    y_full = convolve(señal, kernel, mode="same")

    # Preparación para animación
    frames = []
    L = len(señal)
    K = len(kernel)
    kflip = kernel[::-1]

    # Índices de soporte equivalentes a modo 'same' con padding implícito
    pad = K // 2

    # Prealoca salida incremental
    y_inc = np.zeros_like(señal)

    # Recorre cada posición del centro del kernel
    for c in range(L):
        # Toma ventana con padding (c - pad ... c + pad)
        i0 = c - pad
        i1 = c + pad + 1
        ventana = np.zeros(K, dtype=np.float32)
        # Copia la parte válida desde la señal
        j0 = max(0, i0)
        j1 = min(L, i1)
        w0 = max(0, -i0)
        w1 = w0 + (j1 - j0)
        if j1 > j0:
            ventana[w0:w1] = señal[j0:j1]
        # Valor de salida en c
        y_val = np.dot(ventana, kflip)
        y_inc[c] = y_val

        # --- Dibujo ---
        fig, axs = plt.subplots(2, 1, figsize=(7.2, 5.2), dpi=dpi, constrained_layout=True)

        # Arriba: señal y kernel sobrepuesto (escalado para visualizar)
        axs[0].plot(señal, lw=2, label="Señal x[n]")
        # kernel reescalado y centrado en c
        k_vis = kflip / (np.max(np.abs(kflip)) + 1e-12)  # [-1,1]
        k_vis = 0.35 * k_vis  # escala vertical
        k_line = np.zeros_like(señal)
        k_start = c - pad
        for kk in range(K):
            idx = k_start + kk
            if 0 <= idx < L:
                k_line[idx] = k_vis[kk] + señal[idx]  # superponer encima de x[n]
        axs[0].plot(k_line, lw=2, alpha=0.9, label="Kernel (volteado) centrado")
        axs[0].axvline(c, ls="--", lw=1)
        axs[0].set_title("Convolución 1D: ventana deslizante")
        axs[0].set_xlim(0, L-1)
        axs[0].legend(loc="upper right")
        axs[0].grid(alpha=0.25)

        # Abajo: salida acumulada y total de referencia
        axs[1].plot(y_full, lw=1.5, alpha=0.45, label="Salida completa (referencia)")
        axs[1].plot(y_inc, lw=2.2, label="Salida acumulada $y[n]$")
        axs[1].set_xlim(0, L-1)
        axs[1].set_title("Salida de la convolución (acumulada)")
        axs[1].legend(loc="upper right")
        axs[1].grid(alpha=0.25)

        frame = fig_to_rgb_array(fig)
        plt.close(fig)
        frames.append(frame)

    imageio.mimsave(archivo_salida, frames, duration=1.0/fps, loop=0)
    print(f"[OK] GIF 1D guardado en: {archivo_salida}")

# ---------- GIF de Convolución 2D ----------
def generar_gif_convolucion_2d(
    H=48,
    W=48,
    ksize=7,
    step=2,
    fps=16,
    archivo_salida="convolucion_2d.gif",
    dpi=120,
):
    """
    Visualiza una imagen sintética, el barrido del kernel y la salida acumulada.
    - step: tamaño del paso del barrido para reducir número de frames.
    """
    # Imagen 2D sintética: patrón tipo "checkerboard" + blobs
    yy, xx = np.meshgrid(np.arange(H), np.arange(W), indexing="ij")
    tablero = (((xx // 6) % 2) ^ ((yy // 6) % 2)).astype(np.float32)
    blob1 = np.exp(-((xx-14)**2 + (yy-18)**2)/(2*5.0**2))
    blob2 = np.exp(-((xx-34)**2 + (yy-30)**2)/(2*7.0**2))
    img = normalize01(0.6*tablero + 0.9*blob1 + 0.8*blob2)

    # Kernel 2D: Gaussiano suave (simétrico y normalizado)
    g1d = np.hanning(ksize)
    g2d = np.outer(g1d, g1d)
    kernel = g2d / (np.sum(g2d) + 1e-12)

    # Convolución completa (para escala/guía)
    y_full = convolve2d(img, kernel, mode="same", boundary="fill", fillvalue=0.0)
    y_full_n = normalize01(y_full)

    # Preparación para animación
    frames = []
    pad = ksize // 2
    padded = np.pad(img, pad_width=pad, mode="constant", constant_values=0.0)
    out = np.zeros_like(img, dtype=np.float32)

    # Barrido fila-columna con paso 'step'
    H_range = range(0, H, step)
    W_range = range(0, W, step)

    for r in H_range:
        for c in W_range:
            # Ventana y valor de salida en (r,c)
            ventana = padded[r:r+ksize, c:c+ksize]
            val = np.sum(ventana * kernel[::-1, ::-1])  # conv: kernel volteado
            out[r, c] = val

            # Dibujo
            fig, axs = plt.subplots(1, 2, figsize=(8.6, 4.6), dpi=dpi, constrained_layout=True)

            # Izquierda: imagen con ventana actual
            axs[0].imshow(img, vmin=0, vmax=1, interpolation="nearest")
            axs[0].set_title("Entrada: imagen 2D\n(Ventana del kernel)")
            rect = Rectangle(
                (max(c-pad, 0), max(r-pad, 0)),
                width=min(ksize, W - max(c-pad, 0)),
                height=min(ksize, H - max(r-pad, 0)),
                fill=False, linewidth=2
            )
            axs[0].add_patch(rect)
            axs[0].set_xticks([]); axs[0].set_yticks([])

            # Derecha: salida acumulada con referencia semitransparente
            out_n = normalize01(out)
            axs[1].imshow(y_full_n, vmin=0, vmax=1, interpolation="nearest", alpha=0.35)
            im = axs[1].imshow(out_n, vmin=0, vmax=1, interpolation="nearest")
            axs[1].set_title("Salida de la convolución (acumulada)")
            axs[1].set_xticks([]); axs[1].set_yticks([])

            # Barra de color pequeña
            cbar = plt.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)
            cbar.set_label("Intensidad (norm.)", rotation=90)

            frame = fig_to_rgb_array(fig)
            plt.close(fig)
            frames.append(frame)

    imageio.mimsave(archivo_salida, frames, duration=1.0/fps, loop=0)
    print(f"[OK] GIF 2D guardado en: {archivo_salida}")

# ---------- Main ----------
if __name__ == "__main__":
    # Ajusta parámetros si deseas GIFs más/menos fluidos o ligeros
    generar_gif_convolucion_1d(
        n=140,
        fps=20,
        archivo_salida="convolucion_1d.gif",
        dpi=120
    )

    generar_gif_convolucion_2d(
        H=48, W=48,
        ksize=7,
        step=2,          # aumenta para menos frames (p.ej., 3 o 4)
        fps=16,
        archivo_salida="convolucion_2d.gif",
        dpi=120
    )


[OK] GIF 1D guardado en: convolucion_1d.gif


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import imageio.v2 as imageio
import matplotlib
matplotlib.use("Agg")  # WSL/headless
import matplotlib.pyplot as plt

IMG_SIZE = 28
KERNEL_NAME = "edge"   # "blur" | "sharpen" | "edge" | "gauss"
K = 5
STRIDE = 2
FPS = 12
OUT_GIF = "convolution_demo.gif"
CMAP = "gray"
FIG_DPI = 120

def make_test_image(n=IMG_SIZE, seed=42):
    rng = np.random.default_rng(seed)
    img = np.zeros((n, n), dtype=float)
    y = np.linspace(0, 1, n).reshape(n, 1)
    x = np.linspace(0, 1, n).reshape(1, n)
    img += 0.25 * (x + y)
    s = n // 3
    img[n//4:n//4 + s, n//5:n//5 + s] += 0.6
    for r in range(3, n, 6):
        img[r:r+1, :] += 0.25
    for c in range(5, n, 7):
        img[:, c:c+1] += 0.25
    img += 0.05 * rng.standard_normal((n, n))
    return np.clip(img, 0, 1)

def make_kernel(name="edge", k=5):
    assert k % 2 == 1, "El kernel debe ser impar."
    if name == "blur":
        ker = np.ones((k, k), dtype=float) / (k * k)
    elif name == "sharpen":
        ker = -np.ones((k, k), dtype=float); ker[k//2, k//2] = (k * k); ker /= (k * k)
    elif name == "edge":
        ker = -np.ones((k, k), dtype=float); ker[k//2, k//2] = (k * k - 1); ker /= (k * k)
    elif name == "gauss":
        sigma = k / 3.0
        ax = np.arange(-(k//2), k//2 + 1)
        xx, yy = np.meshgrid(ax, ax)
        ker = np.exp(-(xx**2 + yy**2) / (2 * sigma**2)); ker /= ker.sum()
    else:
        raise ValueError("Kernel no reconocido.")
    return ker

def fig_to_rgb_array(fig):
    # Compatibilidad Matplotlib >= 3.9 (usa buffer_rgba)
    fig.canvas.draw()
    buf = np.asarray(fig.canvas.buffer_rgba())   # (H, W, 4) RGBA
    return buf[..., :3].copy()                   # descartar alfa

def convolve_and_frames(img, ker, stride=1):
    H, W = img.shape
    kH, kW = ker.shape
    outH = (H - kH) // stride + 1
    outW = (W - kW) // stride + 1

    out = np.zeros((outH, outW), dtype=float)
    frames = []

    fig = plt.figure(figsize=(8, 3), dpi=FIG_DPI)

    for i_out, i in enumerate(range(0, H - kH + 1, stride)):
        for j_out, j in enumerate(range(0, W - kW + 1, stride)):
            patch = img[i:i + kH, j:j + kW]
            val = float(np.sum(patch * ker))
            out[i_out, j_out] = val

            plt.clf()
            ax1 = plt.subplot(1, 3, 1)
            ax1.set_title("Entrada")
            ax1.imshow(img, cmap=CMAP, vmin=0, vmax=1)
            rect = matplotlib.patches.Rectangle((j - 0.5, i - 0.5), kW, kH,
                                                fill=False, linewidth=2, edgecolor="red")
            ax1.add_patch(rect); ax1.set_xticks([]); ax1.set_yticks([])

            ax2 = plt.subplot(1, 3, 2)
            ax2.set_title("Kernel")
            kmax = np.max(np.abs(ker))
            ax2.imshow(ker, cmap="bwr", vmin=-kmax, vmax=kmax)
            for r in range(kH):
                for c in range(kW):
                    ax2.text(c, r, f"{ker[r,c]:.2f}", ha="center", va="center", fontsize=7)
            ax2.set_xticks([]); ax2.set_yticks([])

            ax3 = plt.subplot(1, 3, 3)
            ax3.set_title("Salida (acumulada)")
            shown = out.copy()
            lo, hi = np.percentile(shown, [5, 95])
            if hi - lo < 1e-6:
                lo, hi = shown.min(), shown.max() + 1e-6
            ax3.imshow(np.clip((shown - lo) / (hi - lo), 0, 1), cmap=CMAP, vmin=0, vmax=1)
            ax3.set_xticks([]); ax3.set_yticks([])
            ax3.text(0.02, 0.95, f"pos=({i},{j}) → {val:.3f}", transform=ax3.transAxes,
                     fontsize=8, bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))

            plt.tight_layout()
            frames.append(fig_to_rgb_array(fig))

    plt.close(fig)
    return frames

def main():
    img = make_test_image(IMG_SIZE)
    ker = make_kernel(KERNEL_NAME, K)
    frames = convolve_and_frames(img, ker, stride=STRIDE)
    imageio.mimsave(OUT_GIF, frames, fps=FPS, loop=0)
    print(f"Listo. GIF guardado en: {OUT_GIF}")
    print(f"Imagen: {IMG_SIZE}x{IMG_SIZE} | Kernel: {KERNEL_NAME} ({K}x{K}) | Stride: {STRIDE} | Frames: {len(frames)}")

if __name__ == "__main__":
    main()


Listo. GIF guardado en: convolution_demo.gif
Imagen: 28x28 | Kernel: edge (5x5) | Stride: 2 | Frames: 144
